In [1]:
%env INSTANCE=mysql

env: INSTANCE=mysql


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/home/jupyter/us-500.csv')
for col in list(df.columns):
    print(col, "max length is", df[col].astype(str).str.len().max())

first_name max length is 10
last_name max length is 13
company_name max length is 30
address max length is 31
city max length is 19
county max length is 20
state max length is 2
zip max length is 5
phone1 max length is 12
phone2 max length is 12
email max length is 34
web max length is 42


In [4]:
!mysql < /home/jupyter/create_table.sql

In [5]:
!mysql load_testing -e "show tables"

+------------------------+
| Tables_in_load_testing |
+------------------------+
| Person                 |
+------------------------+


In [6]:
!mysql load_testing -e "desc Person"

+------------+-------------+------+-----+---------+----------------+
| Field      | Type        | Null | Key | Default | Extra          |
+------------+-------------+------+-----+---------+----------------+
| id         | int         | NO   | PRI | NULL    | auto_increment |
| first_name | varchar(15) | NO   |     | NULL    |                |
| last_name  | varchar(18) | NO   |     | NULL    |                |
| company    | varchar(35) | YES  |     | NULL    |                |
| address    | varchar(41) | YES  |     | NULL    |                |
| city       | varchar(24) | YES  |     | NULL    |                |
| county     | varchar(25) | YES  |     | NULL    |                |
| state_prov | varchar(20) | YES  |     | NULL    |                |
| ZIP        | char(5)     | YES  |     | NULL    |                |
| ph1        | char(12)    | YES  |     | NULL    |                |
| ph2        | char(12)    | YES  |     | NULL    |                |
| email      | varchar(44) | YES  

In [ ]:
## Please note, we got errors for address and email fields being too long when loading the large file, so we added 10 to those 
## fields instead of 5

In [32]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
%%timeit
import mysql.connector

connection = mysql.connector.connect(
  host="10.128.0.4",
  user="root",
  password="zwne",
  database="load_testing",
  autocommit=False
)

print(connection)

df2 = pd.read_csv('us-1000000.csv') 
rows = df2.values.tolist()
count = 0
total = 0

try:
    for row in rows:
        sql = "INSERT INTO Person (first_name, last_name, company, address, city, county, state_prov, zip, ph1, ph2, email, web)\
          VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        rec = row
    
        cursor = connection.cursor()
        cursor.execute(sql, rec)
        
        count += 1
        total += 1
        
        if count == 5000:
            connection.commit()
            print(total, "records inserted successfully into Person table")
            count =0
            cursor.close()
            
        
    
except mysql.connector.Error as error:
        print("Failed to insert record into Person table {}".format(error))

finally:
    if connection.is_connected():
        connection.close()
        print("MySQL connection is closed")
            

5000 records inserted successfully into Person table
10000 records inserted successfully into Person table
15000 records inserted successfully into Person table
20000 records inserted successfully into Person table
25000 records inserted successfully into Person table
30000 records inserted successfully into Person table
35000 records inserted successfully into Person table
40000 records inserted successfully into Person table
45000 records inserted successfully into Person table
50000 records inserted successfully into Person table
55000 records inserted successfully into Person table
60000 records inserted successfully into Person table
65000 records inserted successfully into Person table
70000 records inserted successfully into Person table
75000 records inserted successfully into Person table
80000 records inserted successfully into Person table
85000 records inserted successfully into Person table
90000 records inserted successfully into Person table
95000 records inserted succes

In [8]:
!mysql load_testing -e "select count(*) from Person"

+----------+
| count(*) |
+----------+
|  8000000 |
+----------+
